In [ ]:
import os
import collections
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm


# Suppressing tf.hub warnings
tf.get_logger().setLevel("ERROR")

In [ ]:
import pandas as pd
import json
json_file = "/kaggle/input/new-json-file/train_new.json"
data = []
with open(json_file, 'r') as file:
    for line in file:
        data.append(json.loads(line))

train_df = pd.DataFrame(data)


data = []
with open(json_file, 'r') as file:
    for line in file:
        data.append(json.loads(line))

train_df = pd.DataFrame(data)
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df.shape

In [ ]:
train_df = train_df[['question', 'anser', 'Quest_hindi', 'ans_hindi']]
train_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['anser'])
train_df.head()

In [ ]:
import os
import numpy as np

#hindi_ans = np.load("/kaggle/input/text-quest-ans-embed/Hindi_Ans_embd_new.npy").squeeze(axis = 1)
#print(hindi_ans.shape)
hindi_ques = np.load("/kaggle/input/text-quest-ans-embed/Hindi_Quest_embd_new.npy").squeeze(axis = 1)
print(hindi_ques.shape)

#english_ans = np.load("/kaggle/input/text-quest-ans-embed/English_Ans_embd_new.npy").squeeze(axis = 1)
#print(english_ans.shape)
english_ques = np.load("/kaggle/input/text-quest-ans-embed/English_Quest_embd_new.npy").squeeze(axis = 1)
print(english_ques.shape)

audio_feats = np.load("/kaggle/input/audio-stacked/Audio_stacked_array.npy")
print(audio_feats.shape)
video_feats = np.load("/kaggle/input/video-stacked/stacked_array.npy")
print(video_feats.shape)

In [ ]:
from sklearn.model_selection import train_test_split
ques_feats = hindi_ques

X_v, X_testv, y_v, y_testv = train_test_split(video_feats,train_df['label_encoded'], test_size=0.2, random_state=42)
X_a, X_testa, y_a, y_testa = train_test_split(audio_feats,train_df['label_encoded'], test_size=0.2, random_state=42)
X_t, X_testt, y_t, y_testt = train_test_split(ques_feats,train_df['label_encoded'], test_size=0.2, random_state=42)

X_trainv, X_valv, y_trainv, y_valv = train_test_split(X_v,y_v, test_size=0.2, random_state=42)
X_traina, X_vala, y_traina, y_vala = train_test_split(X_a, y_a, test_size=0.2, random_state=42)
X_traint, X_valt, y_traint, y_valt = train_test_split(X_t,y_t, test_size=0.2, random_state=42)

In [ ]:
! pip install keras_nlp

In [ ]:
import keras_nlp
import tensorflow as tf
encoder1 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder2 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder3 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
from keras.layers import Lambda
from keras import backend as K
from numpy import newaxis

In [ ]:
def create_video_encoder():
    input_video = tf.keras.Input(shape=[768, 1])
    x1 = tf.keras.layers.Conv1D(32, 3, activation = 'relu', padding = 'same', name= "cnn1")(input_video)
    x1 = tf.keras.layers.MaxPooling1D(name= "maxpool1")(x1)
    x1 = encoder1(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Dense(120, activation= None)(x1)
    return tf.keras.Model(inputs= input_video, outputs = x1, name = "video_encoder")

encoder_v = create_video_encoder()
print(encoder_v.summary())

In [ ]:
def create_audio_encoder(): 
    input_audio = tf.keras.Input(shape=[768, 1])
    x2 = tf.keras.layers.Conv1D(32, 3, activation = 'relu', padding = 'same')(input_audio)
    x2 = tf.keras.layers.MaxPooling1D()(x2)
    x2 = encoder2(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Dense(120, activation= None)(x2)
    return tf.keras.Model(inputs= input_audio, outputs = x2, name = "audio_encoder")

encoder_a = create_audio_encoder()
print(encoder_a.summary())

In [ ]:
def create_text_encoder():     
    input_text = tf.keras.Input(shape=[1024, 1])
    x3 = tf.keras.layers.Conv1D(32, 3, activation = 'relu', padding = 'same')(input_text)
    x3 = tf.keras.layers.MaxPooling1D()(x3)
    x3 = encoder3(x3)
    x3 = tf.keras.layers.Flatten()(x3)
    x3 = tf.keras.layers.Dense(120, activation= None)(x3)
    return tf.keras.Model(inputs= input_text, outputs = x3, name = "text_encoder")

encoder_t = create_text_encoder()
print(encoder_t.summary())

In [ ]:
# Main reference: https://keras.io/examples/vision/nl_image_search/
# Secondary reference: https://towardsdatascience.com/simple-implementation-of-openai-clip-model-a-tutorial-ace6ff01d9f2

class TripleEncoder(keras.Model):
    def __init__(self, video_encoder, audio_encoder, text_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.video_encoder = video_encoder
        self.audio_encoder = audio_encoder
        self.text_encoder = text_encoder
        self.temperature = temperature
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        video_embeddings = self.video_encoder(features["video"], training=training)
        audio_embeddings = self.audio_encoder(features["audio"], training=training)
        text_embeddings = self.text_encoder(features["text"], training=training)
        return video_embeddings, audio_embeddings, text_embeddings

    def compute_loss(self, embeddings1, embeddings2):
        logits = tf.matmul(embeddings1, embeddings2, transpose_b=True) / self.temperature
        similarity_matrix = tf.matmul(embeddings2, embeddings2, transpose_b=True)
        targets = keras.activations.softmax(similarity_matrix / (2 * self.temperature))
        loss = keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        return loss

    def train_step(self, features):
        with tf.GradientTape() as tape:
            video_embeddings, audio_embeddings, text_embeddings = self(features, training=True)
            video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
            video_text_loss = self.compute_loss(video_embeddings, text_embeddings)
            audio_text_loss = self.compute_loss(audio_embeddings, text_embeddings)
            total_loss = video_audio_loss + video_text_loss + audio_text_loss
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        video_embeddings, audio_embeddings, text_embeddings = self(features, training=False)
        video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
        video_text_loss = self.compute_loss(video_embeddings, text_embeddings)
        audio_text_loss = self.compute_loss(audio_embeddings, text_embeddings)
        total_loss = video_audio_loss + video_text_loss + audio_text_loss
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}


In [ ]:
num_samples = 1000
video_data = np.random.rand(num_samples, 768)
audio_data = np.random.rand(num_samples, 768)
text_data = np.random.rand(num_samples, 1024)

dataset = {
    "video": video_data,
    "audio": audio_data,
    "text": text_data,
}

"""
dataset = {
    "video": X_v,
    "audio": X_a,
    "text": X_t,
}
"""

tf_dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(32)


video_encoder = create_video_encoder()
audio_encoder = create_audio_encoder()
text_encoder = create_text_encoder()


triple_encoder = TripleEncoder(video_encoder, audio_encoder, text_encoder, temperature=1.0)

triple_encoder.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4))


num_epochs = 50
history = triple_encoder.fit(tf_dataset, epochs=num_epochs)


evaluation_result = triple_encoder.evaluate(tf_dataset)
print("Evaluation Loss:", evaluation_result)


In [ ]:
# Getting Downstream features out of a trained encoder

trained_video_encoder = create_video_encoder()
trained_video_encoder.set_weights(triple_encoder.video_encoder.get_weights())
new_video_data = np.random.rand(10, 768)
new_video_features = trained_video_encoder.predict(new_video_data)


print("Extracted Video Features:")
print(new_video_features.shape)

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')

video_features = video_features_df.iloc[:, 1:-5].values  
audio_features = audio_features_df.iloc[:, 1:-5].values 

video_features = (video_features - np.mean(video_features, axis=0)) / np.std(video_features, axis=0)
audio_features = (audio_features - np.mean(audio_features, axis=0)) / np.std(audio_features, axis=0)


split_index = int(0.8 * len(video_features)) 
train_video_features = video_features[:split_index]
test_video_features = video_features[split_index:]

train_audio_features = audio_features[:split_index]
test_audio_features = audio_features[split_index:]

# Creating TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({"video": train_video_features, "audio": train_audio_features}))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices(({"video": test_video_features, "audio": test_audio_features}))
test_dataset = test_dataset.batch(32)

def create_video_encoder():
    input_video = tf.keras.Input(shape=[video_features.shape[1]])  # Assuming shape of video features
    x1 = tf.keras.layers.Reshape((video_features.shape[1], 1))(input_video)  # Adding a channel dimension
    x1 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same', name="cnn1")(x1)
    x1 = tf.keras.layers.MaxPooling1D(name="maxpool1")(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Dense(120, activation=None)(x1)
    return tf.keras.Model(inputs=input_video, outputs=x1, name="video_encoder")

def create_audio_encoder():
    input_audio = tf.keras.Input(shape=[audio_features.shape[1]])  # Assuming shape of audio features
    x2 = tf.keras.layers.Reshape((audio_features.shape[1], 1))(input_audio)  # Adding a channel dimension
    x2 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(x2)
    x2 = tf.keras.layers.MaxPooling1D()(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Dense(120, activation=None)(x2)
    return tf.keras.Model(inputs=input_audio, outputs=x2, name="audio_encoder")

class TripleEncoder(tf.keras.Model):
    def __init__(self, video_encoder, audio_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.video_encoder = video_encoder
        self.audio_encoder = audio_encoder
        self.temperature = temperature
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        video_embeddings = self.video_encoder(features["video"], training=training)
        audio_embeddings = self.audio_encoder(features["audio"], training=training)
        return video_embeddings, audio_embeddings

    def compute_loss(self, embeddings1, embeddings2):
        logits = tf.matmul(embeddings1, embeddings2, transpose_b=True) / self.temperature
        similarity_matrix = tf.matmul(embeddings2, embeddings2, transpose_b=True)
        targets = tf.keras.activations.softmax(similarity_matrix / (2 * self.temperature))
        loss = tf.keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        return loss

    def train_step(self, features):
        with tf.GradientTape() as tape:
            video_embeddings, audio_embeddings = self(features, training=True)
            video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
            total_loss = video_audio_loss
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        video_embeddings, audio_embeddings = self(features, training=False)
        video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
        total_loss = video_audio_loss
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

video_encoder = create_video_encoder()
audio_encoder = create_audio_encoder()

triple_encoder = TripleEncoder(video_encoder, audio_encoder, temperature=1.0)

triple_encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

num_epochs = 50
history = triple_encoder.fit(train_dataset, epochs=num_epochs)

evaluation_result = triple_encoder.evaluate(test_dataset)
print("Evaluation Loss:", evaluation_result)


Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 3.4868
Epoch 2/50
1/1 [==============================] - 0s 49ms/step - loss: 3.6079
Epoch 3/50
1/1 [==============================] - 0s 48ms/step - loss: 0.7586
Epoch 4/50
1/1 [==============================] - 0s 53ms/step - loss: 0.2144
Epoch 5/50
1/1 [==============================] - 0s 53ms/step - loss: 0.0160
Epoch 6/50
1/1 [==============================] - 0s 46ms/step - loss: 0.0017
Epoch 7/50
1/1 [==============================] - 0s 42ms/step - loss: 2.0714e-04
Epoch 8/50
1/1 [==============================] - 0s 29ms/step - loss: 2.8889e-05
Epoch 9/50
1/1 [==============================] - 0s 28ms/step - loss: 4.6062e-06
Epoch 10/50
1/1 [==============================] - 0s 28ms/step - loss: 8.4033e-07
Epoch 11/50
1/1 [==============================] - 0s 30ms/step - loss: 1.7514e-07
Epoch 12/50
1/1 [==============================] - 0s 28ms/step - loss: 4.1352e-08
Epoch 13/50
1/1 [======================